In [1]:
# https://docs.tweepy.org/en/v3.5.0/getting_started.html

In [2]:
# Import keys from file not commited to GitHub to keep my credentials secret
from setup_api import consumerKey, consumerSecret, accessToken, accessTokenSecret

import tweepy
import datetime
import pandas as pd

In [5]:
# https://stackoverflow.com/questions/49731259/tweepy-get-tweets-between-two-dates

# Function to get all tweet ID's from a given date range for a SINGLE USER
def tweets_in_dates(user, startDateStr, endDateStr):
    '''
    user - a twitter user
    startDate - earlist tweet we want in format 'YYYY-MM-DD'
    endDate - most re ent tweet we want in format 'YYYY-MM-DD'
    '''

    # Convert date strings to datetime objects
    startDate = datetime.datetime.strptime(startDateStr, '%Y-%m-%d')
    endDate = datetime.datetime.strptime(endDateStr, '%Y-%m-%d')

    tweets = []
    tmpTweets = api.user_timeline(user)

    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            #tweets.append(tweet) # default
            #tweets.append(tweet.id) #first attempt
            tweets.append([user,tweet.id])


    while (tmpTweets[-1].created_at > startDate):
        tmpTweets = api.user_timeline(user, max_id = tmpTweets[-1].id)
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                #tweets.append(tweet) # default
                #tweets.append(tweet.id) #first attempt
                tweets.append([user,tweet.id])

    return tweets

In [6]:

# Function to get all tweet ID's from a given date range for a list of users

def get_users_tweet_id_in_range(users, startDate, endDate):
    '''
    users - a list of twitter user
    startDate - earlist tweet we want in format 'YYYY-MM-DD'
    endDate - most re ent tweet we want in format 'YYYY-MM-DD'
    '''

    res = []

    for user in users:

        print(f"User: {user} Number: {users.index(user)+1} of {len(users)}")

        # Append results of function to results
        # Extend will insure everying is in the same list
        # Append would create a list of lists
        res.extend( tweets_in_dates(user, startDate, endDate ) )

    #Convert results from list of list to DataFrame
    tweets_df = pd.DataFrame.from_records(res, columns=['User', 'TweetId'])

    return tweets_df


In [7]:
uk_news_users = ['BBC', 'BBCWales', 'BBCEngland', 'BBCScotland', 'ITVWales']

results = get_users_tweet_id_in_range(uk_news_users, '2021-05-20', '2021-05-22')

User: BBC Number: 1 of 5
User: BBCWales Number: 2 of 5
User: BBCEngland Number: 3 of 5
User: BBCScotland Number: 4 of 5
User: ITVWales Number: 5 of 5


In [8]:
results

,User,TweetId
0,BBC,1395726076903526400
1,BBC,1395418580896919554
2,BBC,1395352481400512513
3,BBC,1395348587345707015
4,BBCWales,1395841074246135809
...,...,...
130,ITVWales,1395338845173043201
131,ITVWales,1395338006630916102
132,ITVWales,1395270118402412547
133,ITVWales,1395266195440472068
